In [8]:
import time
import threading
from datetime import datetime
from typing import Dict, Optional
import pandas as pd 
import warnings
warnings.filterwarnings("ignore")

In [9]:
from ibapi.client import EClient   # responsible for making call to API server
from ibapi.wrapper import EWrapper  # handles callback
from ibapi.contract import Contract # how IB defines an instrument
from ibapi.order import Order  # how IB defines an order
from ibapi.common import BarData  # bar of historical data we get back

In [10]:
# Create a class to combine EClient and EWrapper
class TradingApp(EClient, EWrapper):
    
    # Instantiate the class 
    def __init__(self) -> None:
        EClient.__init__(self, self)
        self.data: Dict[int, pd.DataFrame] = {}
        
    def error(self, reqId, errorCode, errorString, advancedOrderRejectJson=None):
        print(f"Error: {reqId}, {errorCode}, {errorString}")
        if advancedOrderRejectJson:
            print(f"Advanced: {advancedOrderRejectJson}")
        
    def get_historical_data(self, reqId: int, contract: Contract) -> pd.DataFrame:
        self.data[reqId] = pd.DataFrame(columns=["time", "high", "low", "close"])
        self.data[reqId].set_index("time", inplace=True)
        self.reqHistoricalData(
            reqId=reqId,
            contract=contract,
            endDateTime="",
            durationStr="1 D",
            barSizeSetting="1 min",
            whatToShow="MIDPOINT",
            useRTH=0,
            formatDate=2,
            keepUpToDate=False,
            chartOptions=[]
        )
        time.sleep(3)
        return self.data[reqId]
    
    def histogramData(self, reqId: int, bar: BarData) -> None:
        df = self.data[reqId]
        df.loc[
            pd.to_datetime(bar.date, unit="s"),
            ["high", "low", "close"]
        ] = [bar.high, bar.low, bar.close]
        df = df.astype(float)
        self.data[reqId] = df
        
    @staticmethod
    def create_contract(symbol: str, secType: str = "STK", exchange: str = "SMART", currency: str = "USD") -> Contract:
        contract = Contract()
        contract.symbol = symbol
        contract.secType = 'STK'
        contract.exchange = 'SMART'
        contract.currency = 'USD'
        return contract

In [11]:
app = TradingApp()

In [12]:
app.connect("127.0.0.1", 7497, clientId=5)

In [13]:
threading.Thread(target=app.run, daemon=True).start()

Error: -1, 2104, Market data farm connection is OK:usfarm.nj
Error: -1, 2104, Market data farm connection is OK:eufarm
Error: -1, 2104, Market data farm connection is OK:cashfarm
Error: -1, 2104, Market data farm connection is OK:usfarm
Error: -1, 2106, HMDS data farm connection is OK:euhmds
Error: -1, 2106, HMDS data farm connection is OK:fundfarm
Error: -1, 2106, HMDS data farm connection is OK:ushmds
Error: -1, 2158, Sec-def data farm connection is OK:secdefeu


Error: 99, 162, Historical Market Data Service error message:No market data permissions for ISLAND STK. Requested market data requires additional subscription for API. See link in 'Market Data Connections' dialog for more details.


In [14]:
nvda = TradingApp.create_contract("NVDA")

In [15]:
data = app.get_historical_data(99, contract=nvda)

In [16]:
print(data)

Empty DataFrame
Columns: [high, low, close]
Index: []
